# MammalianLifespan

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.MammalianLifespan)

class MammalianLifespan(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        """
        Applies an anti-log transformation.
        """
        return torch.exp(x)



In [3]:
model = pya.models.MammalianLifespan()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'mammalianlifespan'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'multi'
model.metadata["year"] = 2023
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Li, Caesar Z., et al. \"Epigenetic predictors of species maximum lifespan and other life history traits in mammals.\" bioRxiv (2023): 2023-11."
model.metadata["doi"] = "https://doi.org/10.1101/2023.11.02.565286"
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download GitHub repository

In [5]:
github_url = "https://github.com/caeseriousli/MammalianMethylationPredictors.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

## Load features

#### From CSV file

In [6]:
df = pd.read_csv('MammalianMethylationPredictors/Predictors/LifespanPredictor_40K_Li2021.csv')
df['feature'] = df['CpG']
df['coefficient'] = df['Coefficient']
df = df[df.Coefficient != 0]

model.features = df['feature'][1:].tolist()

## Load weights into base model

In [7]:
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

#### Linear model

In [8]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [9]:
model.reference_values = [0.5] * len(model.features)

## Load preprocess and postprocess objects

In [10]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [11]:
model.postprocess_name = 'anti_log'
model.postprocess_dependencies = None

## Check all clock parameters

In [12]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Li, Caesar Z., et al. "Epigenetic predictors of species maximum '
             'lifespan and other life history traits in mammals." bioRxiv '
             '(2023): 2023-11.',
 'clock_name': 'mammalianlifespan',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1101/2023.11.02.565286',
 'notes': None,
 'research_only': None,
 'species': 'multi',
 'version': None,
 'year': 2023}
reference_values: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]... [Total elements: 152]
preprocess_name: None
preprocess_dependencies: None
postprocess_name: 'anti_log'
postprocess_dependencies: None
features: ['cg00039845', 'cg00300233', 'cg00810217', 'cg01020408', 'cg01266508', 'cg01309159', 'cg01786675', 'cg02476543', 'cg0257

## Basic test

In [13]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[1.8245e+00],
        [7.4422e-04],
        [1.6053e+00],
        [2.6872e-01],
        [3.1673e-03],
        [2.4528e+01],
        [5.4639e+00],
        [1.6672e-02],
        [1.2177e-01],
        [1.5087e-02]], dtype=torch.float64, grad_fn=<ExpBackward0>)

## Save torch model

In [14]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [15]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted folder: MammalianMethylationPredictors
